# Predictive Variables

In [ ]:
############### CHANGE THESE TO YOUR LIKING ###############

#specie = 'leptocybe-invasa' # 'leptocybe-invasa' # 'thaumastocoris-peregrinus' # 
region = region_train
#region = 'east-asia'  # 'east-asia' # 'south-east-asia' # 
training = True
#savefig = True

###########################################################

In [ ]:
import os

import numpy as np
import xarray as xr
import rioxarray

import pandas as pd
import geopandas as gpd
from geocube.api.core import make_geocube

import matplotlib.pyplot as plt
import matplotlib as mpl

import cartopy.io.shapereader as shapereader

params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

In [ ]:
# data_path = os.path.join(os.sep, 'scratch', 'aciar-fst', 'data')
data_path = os.path.join(os.path.dirname(os.getcwd()), 'data')
docs_path = os.path.join(os.path.dirname(os.getcwd()), 'docs')
figs_path = os.path.join(os.path.dirname(os.getcwd()), 'figs')
out_path = os.path.join(os.path.dirname(os.getcwd()), 'out', specie)
input_path = os.path.join(out_path, 'input', 'train') if training else os.path.join(out_path, 'input', 'test')

print(input_path)

In [ ]:
regions = {
    'east-asia': ['China', 'Taiwan', 'Japan', 'North Korea', 'South Korea'],
    'south-east-asia': ['Brunei', 'Myanmar', 'Cambodia', 'East Timor', 'Indonesia', 'Laos', 'Malaysia', 'Philippines', 'Singapore', 'Thailand', 'Vietnam'],
    'australia': ['Australia'],
    'australasia' : ['Australia', 'New Zealand'],
    'india-sri-lanka' : ['India','Sri Lanka'],
    'all' : ['Australia','France','Italy','Portugal','South Africa','United States of America','Madagascar','Spain','Greece','Cyprus','Mexico','Kenya','Algeria','Israel','Egypt','Ethiopia','Ghana','Malawi','Mauritius','Morocco','Mozambique','Rwanda','Sierra Leone','United Republic of Tanzania','Tunisia','Uganda','Zimbabwe','China','India','Iran','Iraq','Jordan','Sri Lanka','Syria','Taiwan','Turkey','Malta','Montenegro','United Kingdom','Argentina','Brazil','Chile','Paraguay','Uruguay']
}

In [ ]:
# Load shapefiles of countries
gdf_countries = gpd.read_file(os.path.join(input_path, '%s.shp' %region))
shapes = [shape for shape in gdf_countries.geometry]

## 1. Elevation

https://www.ncei.noaa.gov/products/etopo-global-relief-model


In [ ]:
# etopo = rioxarray.open_rasterio(os.path.join(data_path, 'topo', 'ETOPO_2022_v1_60s_N90W180_bed.tif'), masked=True)
# etopo_east_asia = etopo.rio.clip(geometries=shapes)
# etopo_east_asia.rio.to_raster(os.path.join('output', 'rasters', 'etopo_east_asia.tif'))

In [ ]:
elev_worldclim = os.path.join(data_path, 'worldclim', 'wc2.1_10m_elev.tif')
srtm = rioxarray.open_rasterio(elev_worldclim, masked=True)
srtm_region = srtm.rio.clip(geometries=shapes)
# srtm_east_asia[np.where(srtm_east_asia < 0)].mask = True
srtm_region.name = 'elevation'
srtm_region.rio.to_raster(os.path.join(input_path, 'srtm_%s.tif' %region))

print('srtm region save')

In [ ]:
# 0.21875*8000
vmax = 0.85*srtm_region.max()
vmin = np.round((vmax - (1.21875 * vmax)))

fig, ax = plt.subplots(figsize=(18,6))
gdf_countries.plot(ax=ax, facecolor='lightgray', edgecolor='k')
# etopo_east_asia.plot(cmap=plt.cm.terrain, vmin=-2000, vmax=8000)
srtm_region.plot(ax=ax, cmap=plt.cm.gist_earth, vmin=vmin, vmax=vmax)
# ax.set_title('Elevation')


In [ ]:
if savefig:
    fig.savefig(os.path.join(figs_path, '03_wordclim_elevation_%s.png' %region), transparent=True)

In [ ]:
# fig, ax = plt.subplots(figsize=(8,6))

# gdf_countries.plot(ax=ax, facecolor='lightgray', edgecolor='k')
# pcol = srtm_region.plot(ax=ax, cmap=plt.cm.gist_earth, vmin=vmin, vmax=vmax, add_colorbar=False, add_labels=False)
# fig.colorbar(pcol, fraction=0.02)

# ax.axis('off')
# ax.set_title('Elevation')

# fig.savefig(os.path.join(docs_path, '03_elevation_%s.png' %region), transparent=True, dpi=600)


## 2. Normalized Difference Vegetation Index (NDVI)
https://land.copernicus.eu/global/products/ndvi

In [ ]:
# https://land.copernicus.vgt.vito.be/manifest/ndvi_stats_all/manifest_cgls_ndvi_stats_all_latest.txt
# ndvi_nc_file = os.path.join(data_path, 'ndvi', 'c_gls_NDVI-STS_2015-2019-0611_GLOBE_PROBAV_V3.0.1.nc')
ndvi_nc_file = os.path.join(data_path, 'ndvi', 'c_gls_NDVI-LTS_1999-2019-1221_GLOBE_VGT-PROBAV_V3.0.1.nc')
ndvi_nc = xr.open_dataset(ndvi_nc_file)

print('nvdi_nc loaded')

In [ ]:
ndvi_nc.rio.write_crs("epsg:4326", inplace=True)
ndvi_nc_region = ndvi_nc.rio.clip(geometries=shapes)

In [ ]:
ndvi_median_region = ndvi_nc_region['median']

ndvi_class_bins = [-np.inf, 0, 0.1, 0.25, 0.4, np.inf]
ndvi_class = np.digitize(ndvi_median_region, ndvi_class_bins)

# Apply the nodata mask to the newly classified NDVI data
ndvi_class = np.ma.masked_where(np.isnan(ndvi_median_region), ndvi_class)

ndvi_median_class_region = xr.DataArray(ndvi_class, 
                                        coords={'lat': ndvi_median_region.lat,'lon': ndvi_median_region.lon,'crs': ndvi_median_region.crs}, 
                                        dims=["lat", "lon"])
ndvi_median_class_region.name = 'ndvi_median_classed'
ndvi_median_class_region.rio.to_raster(os.path.join(input_path, 'ndvi-median-classed_%s.tif' %region))

print('ndvi_median saved')

In [ ]:
col_dict = {0: ['white', 'Unknown'],
            1: ['gray', 'No Vegetation'],
            2: ['y', 'Bare Area'],
            3: ['yellowgreen', 'Low Vegetation'],
            4: ['g', 'Moderate Vegetation'],
            5: ['darkgreen', 'High Vegetation']}

cmap = mpl.colors.ListedColormap([col_dict[x][0] for x in col_dict.keys()])
labels = [col_dict[x][1] for x in col_dict.keys()]
cmap.set_bad='k'

norm = mpl.colors.BoundaryNorm(np.arange(-0.5,6), cmap.N) 
fmt = mpl.ticker.FuncFormatter(lambda x, pos: labels[norm(x)])

In [ ]:
# fig, ax = plt.subplots(ncols=2, figsize=(18, 6))

# gdf_countries.plot(ax=ax[0], facecolor='lightgray', edgecolor='k')
# pcol = ndvi_median_region.plot(ax=ax[0], cmap=plt.cm.YlGn, vmin=0, vmax=1, add_colorbar=False)
# plt.colorbar(pcol, extend='both')
# ax[0].set_title('Normalized Difference Vegetation Index')

# gdf_countries.plot(ax=ax[1], facecolor='lightgray', edgecolor='k')
# pcol = ndvi_median_class_region.plot(ax=ax[1], cmap=cmap, norm=norm, add_colorbar=False)
# plt.colorbar(pcol, format=fmt, ticks=np.linspace(0,5,6))
# ax[1].set_title('NDVI subdivided in classes')

# # Auto adjust subplot to fit figure size
# plt.tight_layout()

In [ ]:
# if savefig:
#     fig.savefig(os.path.join(figs_path, '03_ndvi_%s.png' %region), transparent=True)

In [ ]:
# fig, ax = plt.subplots(figsize=(8,6))

# gdf_countries.plot(ax=ax, facecolor='lightgray', edgecolor='k')
# # pcol = ndvi_median_region.plot(ax=ax, cmap=plt.cm.YlGn, vmin=0, vmax=1, add_colorbar=False)
# # fig.colorbar(pcol, fraction=0.02)

# pcol = ndvi_median_class_region.plot(ax=ax, cmap=cmap, norm=norm, add_colorbar=False)
# fig.colorbar(pcol, format=fmt, ticks=np.linspace(0,5,6), fraction=0.02)

# ax.set_title('NDVI subdivided in classes')
# ax.axis('off')
# fig.savefig(os.path.join(docs_path, '03_ndvi_%s.png' %region), transparent=True, dpi=600)


## 3. Bioclim variables (WorldClim)

In [ ]:
bioclim_names = {
    1: 'Annual Mean Temperature',
    2: 'Mean Diurnal Range (Mean of monthly (max temp - min temp))',
    3: 'Isothermality (BIO2/BIO7) (×100)',
    4: 'Temperature Seasonality (standard deviation ×100)',
    5: 'Max Temperature of Warmest Month',
    6: 'Min Temperature of Coldest Month',
    7: 'Temperature Annual Range (BIO5-BIO6)',
    8: 'Mean Temperature of Wettest Quarter',
    9: 'Mean Temperature of Driest Quarter',
    10: 'Mean Temperature of Warmest Quarter',
    11: 'Mean Temperature of Coldest Quarter',
    12: 'Annual Precipitation',
    13: 'Precipitation of Wettest Month',
    14: 'Precipitation of Driest Month',
    15: 'Precipitation Seasonality (Coefficient of Variation)',
    16: 'Precipitation of Wettest Quarter',
    17: 'Precipitation of Driest Quarter',
    18: 'Precipitation of Warmest Quarter',
    19: 'Precipitation of Coldest Quarter'
}

In [ ]:
bioclim_no = list(np.arange(1,20))#[1, 5, 6, 12, 13, 14]
file_names_worldclim = []
for no in bioclim_no:
    file_names_worldclim.append('wc2.1_10m_bio_%s.tif' %no) 

In [ ]:
rasters, labels = [], []
for no, file_name in zip(bioclim_no, file_names_worldclim):

    worldclim = rioxarray.open_rasterio(os.path.join(data_path, 'worldclim', file_name), masked=True)
    wc_region = worldclim.rio.clip(geometries=shapes)
    wc_region.name = bioclim_names[no]
    wc_region.rio.to_raster(os.path.join(input_path, 'wc2.1_10m_bio_%s_%s.tif' %(no, region))) #'wc2.1_10m_bio-%02d_%s.tif'
    
    rasters.append('wc2.1_10m_bio_%s_%s.tif' %(no, region))
    labels.append('bioclim-%02d' %no)

titles = ['bio-%02d: %s' %(no, name) for (no, name) in bioclim_names.items()]
# rasters, labels, titles

In [ ]:
# bioclim_no = list(np.arange(1,20))#[1, 5, 6, 12, 13, 14]
# file_names_worldclim = []
# for no in bioclim_no:
#     file_names_worldclim.append('bio_%s_from_era5land.tif' %no) 

# rasters, labels = [], []
# for no, file_name in zip(bioclim_no, file_names_worldclim):

#     worldclim = rioxarray.open_rasterio(os.path.join('/mnt/bweeding_workspace/ACIAR/bioclim_from_era/', file_name), masked=True)
#     wc_region = worldclim.rio.clip(geometries=shapes)
#     wc_region.name = bioclim_names[no]
#     wc_region.rio.to_raster(os.path.join(input_path, 'wc2.1_10m_bio_%s_%s.tif' %(no, region))) #'wc2.1_10m_bio-%02d_%s.tif'
    
#     rasters.append('wc2.1_10m_bio_%s_%s.tif' %(no, region))
#     labels.append('bioclim-%02d' %no)

# titles = ['bio-%02d: %s' %(no, name) for (no, name) in bioclim_names.items()]

In [ ]:
# bioclim_plot = input('Provide bioclim numbers to plot (max of 9):').split()
# bioclim_plot = [int(bc) for bc in bioclim_plot]
bioclim_plot = [1, 5, 6, 12, 13, 14]

In [ ]:
if len(bioclim_plot) <= 3:
    nrows, ncols = 1, len(bioclim_plot)
    figsize = (18, 4)
elif 3 < len(bioclim_plot) <= 6:
    nrows, ncols = 2, 3
    figsize = (18, 7)
elif 6 < len(bioclim_plot) <= 9:
    nrows, ncols = 3, 3
    figsize = (18, 10)


fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, sharex=True, sharey=True)
for iax, bc_no in enumerate(bioclim_plot):
    r, c = iax // ncols, iax % ncols
    axes = ax[c] if nrows == 1 else ax[r, c]
    ds = rioxarray.open_rasterio(os.path.join(input_path, rasters[bc_no - 1]), masked=True)
    if bc_no < 12:
        cmap = plt.cm.Spectral_r
    elif bc_no == 12:
        cmap = plt.cm.BuGn
    else:
        cmap = plt.cm.BrBG
    gdf_countries.plot(ax=axes, facecolor='lightgray', edgecolor='k')
#     pcol = ds.plot(ax=ax[r, c], cmap=attrs[titles[iax]][0], vmin=attrs[titles[iax]][1], vmax=attrs[titles[iax]][2], add_colorbar=False, add_labels=False)
    pcol = ds.plot(ax=axes, cmap=cmap, add_colorbar=False, add_labels=False)
    axes.set_title(titles[bc_no - 1])
    
    plt.colorbar(pcol, extend='both')

plt.tight_layout()

In [ ]:
if savefig:
    fig.savefig(os.path.join(figs_path, '03_wordclim_bioclim_%s.png' %region), transparent=True)

In [ ]:
# bc_no = 14
# cmap = plt.cm.Spectral_r if bc_no < 12 else plt.cm.BuGn
# fig, ax = plt.subplots(figsize=(8,6))

# ds = rioxarray.open_rasterio(os.path.join(input_path, rasters[bc_no - 1]), masked=True)
# gdf_countries.plot(ax=ax, facecolor='lightgray', edgecolor='k')
# ds.plot(ax=ax, cmap=cmap, add_colorbar=False, add_labels=False)
# # ax.set_title(titles[bc_no - 1])

# ax.axis('off')
# fig.savefig(os.path.join(docs_path, '04_bioclim-%s_%s.png' %(bc_no, region)), transparent=True, dpi=600)

In [ ]:
# Percent Tree Cover (PTC)